In [27]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

True

In [4]:
llm = ChatOpenAI(model="gpt-4-turbo-preview")

In [7]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent",),
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(
            class_=(
                "post-content",
                "post-title",
                "post-header",
            ),
        ),
    },
)

docs = loader.load()

In [18]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

splits = text_splitter.split_documents(docs)

In [21]:
embedding = OpenAIEmbeddings()

vectorstore = ElasticsearchStore.from_documents(
    documents=splits,
    embedding=embedding,
    index_name="qna_with_rag",
    es_url=os.getenv("ELASTICSEARCH_URL"),
)

In [29]:
retriever = vectorstore.as_retriever()

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            (
                "You are an assistant for question-answering tasks. "
                "Use the following pieces of retrieved context to answer the question. "
                "If you don't know the answer, just say that you don't know. "
                "Use three sentences maximum and keep the answer concise.\n\n"
                "Question: {input}\n\n"
                "Context: {context}\n\n"
                "Answer:"
            ),
        )
    ]
)

In [25]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [30]:
chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [32]:
chain.invoke("작업 분해가 뭐야?")

'작업 분해는 복잡한 작업을 더 작고 간단한 단계로 나누는 과정입니다. 이를 통해 큰 작업을 여러 관리 가능한 작업으로 변환하고 모델의 사고 과정을 해석하는 데 도움을 줍니다. 작업 분해는 간단한 프롬프팅, 특정 지시사항 사용, 또는 인간의 입력을 통해 수행될 수 있습니다.'

In [36]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_question_system_prompt = (
    "대화 이력을 참조하는 마지막 사용자의 질문과 대화 이력이 주어졌을 때, 대화 이력 없이도 이해할 수 있는 질문을 구성하라."
    # Given a chat history and the latest user question which might reference context in the chat history,
    # formulate a standalone question which can be understood without the chat history.
    "질문에 대한 답변은 하지 않는다. 지시대로 질문을 재구성하거나, 질문 그대로 반환하라."
    # Do not answer the question, just reformulate it if needed and otherwise return it as is.
)

contextualize_question_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_question_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    contextualize_question_prompt,
)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

question_answer_system_prompt = (
    "너는 질답 작업을 위한 어시스턴트이다. "
    # You are an assisstant for question-answering tasks.
    "검색된 맥락을 사용하여 질문에 답변하라. "
    # Use the following pieces of retrieved context to answer the question.
    "만약 답변할 수 없는 경우, 모른다고 답변하라. "
    # If you don't know the answer, just say that you don't know.
    "3 문장 이내로 간결하게 답변하라.\n\n"
    # use three sentences maximum and keep the answer concise.
    "{context}"
)

question_answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", question_answer_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(
    llm,
    question_answer_prompt,
)

rag_chain = create_retrieval_chain(
    history_aware_retriever,
    question_answer_chain,
)

In [43]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

question = "작업 분해가 뭐야?"

answer = rag_chain.invoke(
    {
        "input": question,
        "chat_history": chat_history,
    }
)

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=answer["answer"]),
    ]
)

In [46]:
answer["answer"]

'더 일반적인 작업 분해 기법 중 하나는 "Chain of Thought" (CoT)입니다. CoT는 모델에게 단계별로 생각하도록 지시하여 복잡한 작업을 여러 간단한 단계로 분해하고, 이를 통해 모델의 성능을 향상시키는 방식입니다. 이 방식은 모델의 사고 과정을 해석하는 데 도움을 주며, 복잡한 작업을 효과적으로 해결하는 데 널리 사용됩니다.'

In [44]:
question = "더 일반적인 기법이 뭐야?"

answer = rag_chain.invoke(
    {
        "input": question,
        "chat_history": chat_history,
    }
)

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=answer["answer"]),
    ]
)

In [47]:
answer["answer"]

'더 일반적인 작업 분해 기법 중 하나는 "Chain of Thought" (CoT)입니다. CoT는 모델에게 단계별로 생각하도록 지시하여 복잡한 작업을 여러 간단한 단계로 분해하고, 이를 통해 모델의 성능을 향상시키는 방식입니다. 이 방식은 모델의 사고 과정을 해석하는 데 도움을 주며, 복잡한 작업을 효과적으로 해결하는 데 널리 사용됩니다.'

In [48]:
answer["context"]

[Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent'}),
 Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought 

In [50]:
for chunk in rag_chain.stream(
    {
        "input": question,
        "chat_history": chat_history,
    }
):
    print(chunk)

{'input': '더 일반적인 기법이 뭐야?', 'chat_history': [HumanMessage(content='작업 분해가 뭐야?'), AIMessage(content='작업 분해는 복잡한 작업을 더 작고 관리하기 쉬운 여러 단계나 하위 목표로 나누는 과정입니다. 이를 통해 대규모 문제를 해결하는 데 도움이 되며, 모델이 문제 해결 과정을 단계별로 생각하게 만들어 성능을 향상시킬 수 있습니다. 예를 들어, "Chain of Thought" 방식은 모델이 복잡한 작업을 여러 간단한 단계로 분해하여 접근하도록 합니다.'), HumanMessage(content='더 일반적인 기법이 뭐야?'), AIMessage(content='더 일반적인 작업 분해 기법 중 하나는 "Chain of Thought" (CoT)입니다. CoT는 모델에게 단계별로 생각하도록 지시하여 복잡한 작업을 여러 간단한 단계로 분해하고, 이를 통해 모델의 성능을 향상시키는 방식입니다. 이 방식은 모델의 사고 과정을 해석하는 데 도움을 주며, 복잡한 작업을 효과적으로 해결하는 데 널리 사용됩니다.')]}
{'context': [Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decompositio